In [37]:
import convokit
import numpy as np
import pandas as pd
from convokit import Corpus, Speaker, Utterance, download
from tqdm.notebook import tqdm

In [2]:
corpus = Corpus(filename=download("diplomacy-corpus"))

Dataset already exists at /Users/lynnette/.convokit/downloads/diplomacy-corpus


In [34]:
converted_dict = []

In [26]:
conversation_df = corpus.get_conversations_dataframe().reset_index()
conversation_df

,id,vectors,meta.acl2020_fold
0,Game1-italy-germany,[],Train
1,Game1-austria-italy,[],Train
2,Game1-italy-russia,[],Train
3,Game1-england-italy,[],Train
4,Game1-turkey-italy,[],Train
...,...,...,...
241,Game9-turkey-russia,[],Train
242,Game9-france-russia,[],Train
243,Game9-turkey-england,[],Train
244,Game9-england-france,[],Train


In [23]:
convo = corpus.get_conversation('Game1-italy-germany')
convo.get_utterances_dataframe().reset_index().head(1)

,id,timestamp,text,speaker,reply_to,conversation_id,meta.speaker_intention,meta.receiver_perception,meta.receiver,meta.absolute_message_index,meta.relative_message_index,meta.year,meta.game_score,meta.game_score_delta,meta.deception_quadrant
0,Game1-italy-germany-0,74,Germany!\n\nJust the person I want to speak wi...,italy-Game1,None,Game1-italy-germany,Truth,Truth,germany-Game1,74,0,1901,3,0,Straightforward


In [35]:
for index, row in tqdm(conversation_df.iterrows()):
    convo_id = row['id']
    train_test = row['meta.acl2020_fold']

    convo = corpus.get_conversation(convo_id)
    convo_df = convo.get_utterances_dataframe().reset_index()
    for convo_idx, convo_row in convo_df.iterrows():
        msg_dict = {}
        msg_dict['convo_id'] = convo_id
        msg_dict['train_test'] = train_test
        msg_dict['msg_id'] = convo_row['id']
        msg_dict['timestamp'] = convo_row['timestamp']
        msg_dict['text'] = convo_row['text']
        msg_dict['speaker'] = convo_row['speaker']
        msg_dict['reply_to'] = convo_row['reply_to']
        msg_dict['speaker_intention'] = convo_row['meta.speaker_intention']
        msg_dict['reciever_perception'] = convo_row['meta.receiver_perception']
        msg_dict['reciever'] = convo_row['meta.receiver']
        msg_dict['absolute_message_index'] = convo_row['meta.absolute_message_index']
        msg_dict['relative_message_index'] = convo_row['meta.relative_message_index']
        msg_dict['year'] = convo_row['meta.year']
        msg_dict['game_score'] = convo_row['meta.game_score']
        msg_dict['game_score_delta'] = convo_row['meta.game_score_delta']
        msg_dict['deception_quadrant'] = convo_row['meta.deception_quadrant']
        
        converted_dict.append(msg_dict)

In [38]:
converted_df = pd.DataFrame(converted_dict)
converted_df.head(5)

,convo_id,train_test,msg_id,timestamp,text,speaker,reply_to,speaker_intention,reciever_perception,reciever,absolute_message_index,relative_message_index,year,game_score,game_score_delta,deception_quadrant
0,Game1-italy-germany,Train,Game1-italy-germany-0,74,Germany!\n\nJust the person I want to speak wi...,italy-Game1,None,Truth,Truth,germany-Game1,74,0,1901,3,0,Straightforward
1,Game1-italy-germany,Train,Game1-italy-germany-1,76,"You've whet my appetite, Italy. What's the sug...",germany-Game1,Game1-italy-germany-0,Truth,Truth,italy-Game1,76,1,1901,3,0,Straightforward
2,Game1-italy-germany,Train,Game1-italy-germany-2,86,👍,italy-Game1,Game1-italy-germany-1,Truth,Truth,germany-Game1,86,2,1901,3,0,Straightforward
3,Game1-italy-germany,Train,Game1-italy-germany-3,87,It seems like there are a lot of ways that cou...,germany-Game1,Game1-italy-germany-2,Truth,Truth,italy-Game1,87,3,1901,3,0,Straightforward
4,Game1-italy-germany,Train,Game1-italy-germany-4,89,"Yeah, I can’t say I’ve tried it and it works, ...",italy-Game1,Game1-italy-germany-3,Truth,None,germany-Game1,89,4,1901,3,0,Unknown


In [39]:
converted_df.to_csv('diplomacy_corpus.csv',index=False)